In [85]:
import numpy as np
import pandas as pd
import pylab as pl

from sklearn.linear_model import Ridge

prediction_weights = 'prediction_weights.csv' 
prediction_rf = 'prediction_rf.csv'
prediction_ridge = 'prediction_ridge.csv'

df_weights = pd.read_csv(prediction_weights)
df_rf = pd.read_csv(prediction_rf)
df_ridge = pd.read_csv(prediction_ridge)


In [86]:
df_stack = pd.merge(df_rf, df_weights, on='Id')
df_stack = pd.merge(df_stack, df_ridge, on='Id')
df_stack

,Id,Sales_x,Sales_y,Sales
0,1,4375.986364,4787.007214,4176.768576
1,2,7692.922030,8080.993636,5878.704890
2,3,9594.088910,10346.925177,8064.102102
3,4,8003.769369,7717.238893,9178.314189
4,5,7642.332141,7604.183504,5663.430915
5,6,5864.320298,6030.074381,4060.569046
6,7,7899.962531,8351.561036,9965.522796
7,8,8464.122199,8595.822056,8156.540802
8,9,5870.514289,5877.829757,3238.789198
9,10,5820.336874,6102.277618,5419.328109


In [87]:
rf_mean = df_rf.Sales.mean()
weights_mean = df_weights.Sales.mean()
ridge_mean = df_ridge.Sales.mean()
print('Means', rf_mean, weights_mean, ridge_mean)

rf_coeff_prior = 2.5
weights_coeff_prior = 0.3
ridge_coeff_prior = 0.2
mean = (rf_coeff_prior*rf_mean + weights_coeff_prior*weights_mean + ridge_coeff_prior*ridge_mean)/3.
print('weighted average of means', mean)
max_dev = max([abs(rf_mean-mean), abs(weights_mean-mean), abs(ridge_mean-mean)]) + 1000


rf_coeff = 1 - abs(mean - rf_mean)/max_dev
weights_coeff = 1 - abs(mean - weights_mean)/max_dev
ridge_coeff = 1 - abs(mean - ridge_mean)/max_dev

sum_coeff = rf_coeff + weights_coeff + ridge_coeff

rf_coeff = 3*rf_coeff/sum_coeff
weights_coeff = 3*weights_coeff/sum_coeff
ridge_coeff = 3*ridge_coeff/sum_coeff

rf_coeff = (rf_coeff + rf_coeff_prior)/2
weights_coeff = (weights_coeff + weights_coeff_prior)/2
ridge_coeff = (ridge_coeff + ridge_coeff_prior)/2

print('coeffs', rf_coeff, weights_coeff, ridge_coeff)
print(rf_coeff + weights_coeff + ridge_coeff, ' must be 3')

df_rf.Sales = rf_coeff*df_rf.Sales
df_weights.Sales = weights_coeff*df_weights.Sales
df_ridge.Sales = ridge_coeff*df_ridge.Sales

df_stack = pd.merge(df_rf, df_weights, on='Id')
df_stack = pd.merge(df_stack, df_ridge, on='Id')

df_result = df_stack[['Id']]
df_result['Sales'] = df_stack[['Sales_x', 'Sales_y', 'Sales']].mean(axis=1)
df_result.to_csv('prediction.csv', index=False)
df_result

Means 5884.491628030663 5972.760133056349 24089.834658938686
weighted average of means 7107.0080139271
coeffs 1.9764164235893036 0.8802421100321582 0.14334146637853798
3.0  must be 3


/home/oem/anaconda3/lib/python3.4/site-packages/IPython/kernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Id,Sales
0,1,4487.066927
1,2,7720.103505
2,3,9741.878121
3,4,7975.817631
4,5,7536.585928
5,6,5826.770756
6,7,8131.161350
7,8,8488.068396
8,9,5746.915639
9,10,5883.901851


In [88]:
del df_rf, df_weights, df_ridge, df_stack, df_result